# Clean extracted data
To do:
* Find Names
* Create function getAllCandidats

In [24]:
import json
import pandas as pd
from pathlib import Path
import re
import utils
from dateutil import parser

In [2]:
# Get all files
df = pd.read_csv(Path('../export/dokumente.csv'))

# Only "Protokoll"
df = df[df.dokument_kategorie == 'Protokoll']

df_raete = pd.read_csv(Path('../export/mitglieder.csv'))
df_raete['_name'] = df_raete['vorname'] +' ' + df_raete['name']
df_raete['_name'] = df_raete['_name'].str.lower()


# Some names are written wrong. Some changed their name (marriage). Add
# Aliases to the dataframe

for a in utils.aliases:
    df_raete = df_raete.append({
        '_name': a[0].lower(),
        'name': a[2],
        'vorname': a[1]}, ignore_index=True)

In [3]:
# Remove this
remove = ['Ratsvizepräsidentin', 'Ratspräsidentin', 'Regierungsrätin',
     'Ratspräsident', 'Regierungsrat', 'Ratsvizepräsident', 
    ', Niederhasli.', ', SVP, Zürich.', ', SVP, Dinhard.',
    ', GLP, Dübendorf.', ', SVP, Volketswil.', ', GLP, Dübendorf.',
    ', SVP, Uetikon am See.']

records = []
notfound = []

# Loop file
for i, row in df.iterrows():

    # Create Name and load
    f = Path('../export/extracts/%s' % row['_filename']).with_suffix('.json')
    data = json.load(open(f, encoding='utf-8'))

    filename = Path(f).stem

    for msg in data:
        #print(msg)

        # Replace double space
        msg = msg.replace('  ', ' ')

        if msg.strip() == '':
            continue

        # Remove Strings
        for r in remove:
            msg = msg.replace(r, '')

        msg = msg.strip()

        # Extract Name
        r1 = re.findall(r"[^(:]*", msg)
        if len(r1) > 0:
            name = r1[0].strip()
            r = df_raete[df_raete['_name'] == name.lower()]
            # If name not found, add it to a list for manual check
            if len(r) == 0:
                notfound.append({
                    'name': name,
                    'f': filename,
                })
                
            else:
                # Name found!

                # Clean text
                text = msg
                text = text.replace('-\n', '')
                text = text.replace('\n', ' ')
                text = text.replace('  ', ' ')
                text = text.strip()

                text = re.findall(r".*:(.*)", text)
                if len(text) > 0:
                    text = text[0].strip()

                    records.append({
                        'name': r.iloc[0]['name'],
                        'vorname': r.iloc[0]['vorname'],
                        'text': text,
                        'f': filename
                    })
        else:
            # No Name found
            print('!! No Name found in:\n%s' % msg)

print("finito")

finito


## Join with dataframe and save

In [13]:
#df_mitglied = pd.read_csv(Path('../export/mitglieder.csv'))
df_votum = pd.DataFrame(records)
df_dokumente = pd.read_csv(Path('../export/dokumente.csv'))
df_dokumente['f'] = df_dokumente['_filename'].apply(lambda x: Path(x).stem)


# Join Votum and Dokumente
df_votum = df_votum.merge(df_dokumente[['f', 'sitzung_name', 'sitzung_date', 'sitzung_gremium']], on = 'f')

# Merge names
df_votum['_join'] = df_votum.apply(lambda row: "%s_%s" % (row['name'], row['vorname']), axis=1)
#df_mitglied['_join'] = df_mitglied.apply(lambda row: "%s_%s" % (row['name'], row['vorname']), axis=1)

# Typecast
df_votum['sitzung_date'] = pd.to_datetime(df_votum['sitzung_date'])
#df_mitglied['start'] = pd.to_datetime(df_mitglied['start'])
#df_mitglied['end'] = df_mitglied['end'].str.replace('9999-12-31T00:00:00.000', '2100-01-01', regex=False)
#df_mitglied['end'] = pd.to_datetime(df_mitglied['end'])

In [28]:
df_votum.head()

,name,vorname,text,f,sitzung_name,sitzung_date,sitzung_gremium,_join,geschlecht,jahrgang
0,Egg,Bernhard,"Zuerst wünsche ich Ihnen allen, auch wenn scho...",2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,Egg_Bernhard,m,1959.0
1,Egg,Bernhard,Dann hat mir Toni Schaller (Altkantonsrat Anto...,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,Egg_Bernhard,NaN,NaN
2,Egg,Bernhard,Wir dürfen für den aus dem Kantonsrat zurückge...,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,Egg_Bernhard,NaN,NaN
3,Egg,Bernhard,"Ich bitte, den Gewählten eintreten zu lassen. ...",2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,Egg_Bernhard,NaN,NaN
4,Egg,Bernhard,«Ich gelobe es.»,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,Egg_Bernhard,NaN,NaN


In [34]:
with open(Path('../export/mitglieder.json')) as f:
    mitglieder = json.load(f)

# Typecast
for record in mitglieder:
    for l in record['partei']:
        l['start'] = parser.parse(l['start'])
        l['end'] = parser.parse(l['end'])

    for l in record['funktion']:
        l['start'] = parser.parse(l['start'])
        l['end'] = parser.parse(l['end'])        

# No join, because could be multiple Partys per name (if they switched). Loop and find party
not_found = []

for i, row in df_votum.iterrows():
    record = None
    for r in mitglieder:
        if ("%s_%s" % (r['name'], r['vorname'])) == row['_join']:
            record = r
            break

    if record == None:
        not_found.append(record)
        continue

    # Find party
    party = None
    for l in record['partei']:
        if (l['start'] <= row['sitzung_date']) and ((l['end'] >= row['sitzung_date'])):
            party = l['bezeichnung']

    # Find Funkction
    function = None
    for l in record['funktion']:
        if (l['start'] <= row['sitzung_date']) and ((l['end'] >= row['sitzung_date'])):
            function = l['bezeichnung']            

    df_votum.loc[i, 'partei'] = party
    df_votum.loc[i, 'geschlecht'] = record['geschlecht']
    df_votum.loc[i, 'jahrgang'] = record['jahrgang']
    df_votum.loc[i, 'funktion'] = function

if len(not_found) > 0:
    print("Could not match everything. Have a look at not_found")


In [37]:
df_votum.to_csv(Path('../export/votum.csv'), index=False)
#df_votum.head()

# Depricated

In [5]:

# No join, because could be multiple Partys per name (if they switched). Loop and find party
not_found = []

for i, row in df_votum.iterrows():

    df = df_mitglied[df_mitglied['_join'] == row['_join']]

    if len(df) == 0:
        msg = "%s not found in Memberlist" % row['_join']
        not_found.append({
            'name': row['_join'],
            'sitzung_date': row['sitzung_date'],
        })
        #print(msg)
        #print(row)
        #raise ValueError(msg)
    else:
        # Check if Function
        df_fun = df[
            (df['start'] <= row['sitzung_date']) & (df['end'] >= row['sitzung_date'])
            ]
        function = None
        if len(df_fun) > 0:
            function = df_fun.iloc[0]['funktion']

        df_votum.loc[i, 'partei'] = df.iloc[0].partei
        df_votum.loc[i, 'geschlecht'] = df.iloc[0].geschlecht
        df_votum.loc[i, 'jahrgang'] = df.iloc[0].jahrgang
        df_votum.loc[i, 'funktion'] = function

    #break

#df_votum = df_votum.merge(df_party[['_join', 'geschlecht', 'jahrgang', 'partei']], on = '_join')
if len(not_found) > 0:
    print("Could not match everything. Have a look at not_found")


In [12]:
df_votum.to_csv(Path('../export/votum.csv'), index=False)
df_votum.tail()

,name,vorname,text,f,sitzung_name,sitzung_date,sitzung_gremium,_join,partei,geschlecht,jahrgang,funktion
33045,Röösli,Brigitte,Die Corona-Pandemie prägte 2020 auch die Arbei...,2021-09-27-17fa216f98fd4d38afd5afb53743ec65-332,136. Sitzung,2021-09-27,KR,Röösli_Brigitte,SP,w,1963.0,Mitglied
33046,Fehr,Raffaela,Die IPW kämpfte gegen die Nebenwirkungen der C...,2021-09-27-17fa216f98fd4d38afd5afb53743ec65-332,136. Sitzung,2021-09-27,KR,Fehr_Raffaela,FDP,w,1985.0,NaN
33047,Aeschbacher,Nathalie,In Anbetracht der schwierigen Umstände hat die...,2021-09-27-17fa216f98fd4d38afd5afb53743ec65-332,136. Sitzung,2021-09-27,KR,Aeschbacher_Nathalie,GLP,w,1986.0,NaN
33048,Willi,Wilma,Das Thema «Nachhaltigkeit» wird bei der IPW gr...,2021-09-27-17fa216f98fd4d38afd5afb53743ec65-332,136. Sitzung,2021-09-27,KR,Willi_Wilma,Grüne,w,1960.0,NaN
33049,Berner,Melanie,"Auch die Integrierte Psychiatrie, die IPW, hat...",2021-09-27-17fa216f98fd4d38afd5afb53743ec65-332,136. Sitzung,2021-09-27,KR,Berner_Melanie,AL,w,1981.0,NaN


In [9]:
# Not founds
if len(df_not) > 0:
    df_not = pd.DataFrame(not_found)
    df_not.name.unique()